Show that geomtry id's for geometries are *not* stable when considered
within a single `MultibodyPlant` and `SceneGraph` pairing.

More conceretely, see:
- https://github.com/RobotLocomotion/drake/blob/v1.0.0/geometry/geometry_ids.h#L31-L34
- https://github.com/RobotLocomotion/drake/blob/v1.0.0/common/identifier.cc#L10-L24


In [1]:
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    BodyIndex,
    DiagramBuilder,
    FindResourceOrThrow,
    Parser,
)

In [2]:
iiwa_file = FindResourceOrThrow(
    "drake/manipulation/models/iiwa_description/urdf/"
    "iiwa14_spheres_dense_elbow_collision.urdf"
)

In [3]:
def make_new_iiwa():
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
    Parser(plant).AddModelFromFile(iiwa_file)
    return plant, scene_graph

In [4]:
def get_geometries(plant, scene_graph, bodies):
    inspector = scene_graph.model_inspector()
    geometry_ids = []
    for geometry_id in inspector.GetAllGeometryIds():
        body = plant.GetBodyFromFrameId(inspector.GetFrameId(geometry_id))
        if body in bodies:
            geometry_ids.append(geometry_id)
    return geometry_ids

In [5]:
for i in range(3):
    plant, scene_graph = make_new_iiwa()
    body = plant.GetBodyByName("iiwa_link_5")
    geometries = get_geometries(plant, scene_graph, {body})
    print(geometries)

[<GeometryId value=115>, <GeometryId value=112>, <GeometryId value=109>]
[<GeometryId value=252>, <GeometryId value=249>, <GeometryId value=246>]
[<GeometryId value=389>, <GeometryId value=386>, <GeometryId value=383>]


---

Note that the above geometry id's are *not* the same, even though
they were produced in the same way.